In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/processed_movie_data.csv", delimiter=';', encoding='utf-8')
df

,budget,genres,imdb_id,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,overview_length,sentiment,year,month,weekday,genre_id
0,30000000.0,Animation,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1.0,1.0,1995-10-30,373554033.0,81.0,1.0,7.7,5415.0,303,0.112121,1995,10,0,0
1,65000000.0,Adventure,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,3.0,1.0,1995-12-15,262797249.0,104.0,2.0,6.9,2413.0,395,-0.218750,1995,12,4,1
2,16000000.0,Comedy,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1.0,1.0,1995-12-22,81452156.0,127.0,1.0,6.1,34.0,270,0.600000,1995,12,4,2
3,60000000.0,Action,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,3.0,1.0,1995-12-15,187436818.0,170.0,2.0,7.7,1886.0,335,-0.053571,1995,12,4,3
4,35000000.0,Action,tt0114576,Sudden Death,International action superstar Jean Claude Van...,5.231580,3.0,1.0,1995-12-22,64350171.0,106.0,1.0,5.5,174.0,448,-0.116667,1995,12,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680,2196531.0,Comedy,tt0453365,FC Venus,"Pete is a football enthusiast, who plays as a ...",0.947509,1.0,1.0,2005-12-30,2411594.0,107.0,1.0,5.6,10.0,669,0.102083,2005,12,4,2
4681,12000000.0,Action,tt0479751,சிவாஜி,Corrupt police and politicians target a comput...,1.323587,1.0,1.0,2007-06-14,19000000.0,185.0,2.0,6.9,25.0,117,-0.055556,2007,6,3,3
4682,800000.0,Comedy,tt0933361,Dikari,"The sea, August, interesting and simple people...",0.903061,0.0,0.0,2006-11-23,1328612.0,100.0,1.0,5.8,6.0,378,0.380000,2006,11,3,2
4683,2000000.0,Romance,tt1718881,Про любоff,"У девушки Даши, приехавшей с подругой «покорят...",0.121844,1.0,1.0,2010-09-30,1268793.0,107.0,1.0,4.0,3.0,534,0.000000,2010,9,3,9


In [3]:
from utils.kohonen.kohonen import build_kohonen_net
from utils.kohonen.neuron_neighbours import prop2iters_neighbour_radius, relu_like_neighbour_radius, exp_neighbour_radius
from utils.kohonen.neuron_lr import inv2iter_learning_rate, fixed_learning_rate

In [4]:
cols = ["budget", "popularity", "production_companies", 
        "production_countries", "revenue", "runtime", 
        "spoken_languages", "vote_average", "vote_count",
        "overview_length", "sentiment", "year",
        "month", "weekday"]

In [5]:
seed_range = 256

In [6]:
to_normalize = cols
for i in range(len(to_normalize)):
    df[to_normalize[i]] = (df[to_normalize[i]] - df[to_normalize[i]].mean()) / df[to_normalize[i]].std()
df

,budget,genres,imdb_id,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,overview_length,sentiment,year,month,weekday,genre_id
0,-0.030242,Animation,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",2.297018,-0.888981,-0.467054,1995-10-30,1.813769,-1.351069,-0.528057,1.573598,3.973387,-0.003497,0.312006,-0.291643,0.885889,-2.399597,0
1,0.850490,Adventure,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,1.420062,0.025974,-0.467054,1995-12-15,1.108865,-0.287850,0.534179,0.686010,1.436552,0.570333,-1.197489,-0.291643,1.475559,0.619027,1
2,-0.382535,Comedy,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",-0.919488,-0.888981,-0.467054,1995-12-22,-0.045295,0.775369,-0.528057,-0.201578,-0.573818,-0.209328,2.537798,-0.291643,1.475559,0.619027,2
3,0.724671,Action,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",1.581780,0.025974,-0.467054,1995-12-15,0.629238,2.763126,0.534179,1.573598,0.991211,0.196096,-0.443914,-0.291643,1.475559,0.619027,3
4,0.095577,Action,tt0114576,Sudden Death,International action superstar Jean Claude Van...,-0.675489,0.025974,-0.467054,1995-12-22,-0.154140,-0.195396,-0.528057,-0.867269,-0.455512,0.900909,-0.731766,-0.291643,1.475559,0.619027,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680,-0.729883,Comedy,tt0453365,FC Venus,"Pete is a football enthusiast, who plays as a ...",-1.437329,-0.888981,-0.467054,2005-12-30,-0.548344,-0.149169,-0.528057,-0.756321,-0.594100,2.279350,0.266211,0.343454,1.475559,0.619027,2
4681,-0.483190,Action,tt0479751,சிவாஜி,Corrupt police and politicians target a comput...,-1.370451,-0.888981,-0.467054,2007-06-14,-0.442768,3.456530,0.534179,0.686010,-0.581424,-1.163633,-0.452966,0.470473,-0.293450,-0.135629,3
4682,-0.765025,Comedy,tt0933361,Dikari,"The sea, August, interesting and simple people...",-1.445233,-1.346458,-1.755714,2006-11-23,-0.555237,-0.472758,-0.528057,-0.534424,-0.597480,0.464299,1.534118,0.406964,1.180724,-0.135629,2
4683,-0.734828,Romance,tt1718881,Про любоff,"У девушки Даши, приехавшей с подругой «покорят...",-1.584158,-0.888981,-0.467054,2010-09-30,-0.555617,-0.149169,-0.528057,-2.531497,-0.600015,1.437316,-0.199512,0.661002,0.591054,-0.135629,9


In [7]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
rectangular

0 12699.968800715906
1 12689.746944033583
2 12514.463238842543
14 12455.197131096924
max:  247 12820.447977504879
range:  365.25084640795467


In [8]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
rectangular

0 14763.80951446214
2 14698.020688598748
9 14639.307332188419
102 14611.77732858622
max:  159 15105.824468001152
range:  494.04713941493173


In [9]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
rectangular

0 12648.781271009899
1 12540.929216153147
2 12481.384688160173
13 12453.2195644513
65 12382.18958173669
195 12373.769587951474
max:  239 12754.070242816793
range:  380.3006548653193


In [10]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
rectangular

0 13295.916651175128
2 13125.305598206132
7 13071.059563268183
74 13019.622844735823
214 12987.531282518088
max:  6 13920.124701255594
range:  932.593418737506


In [11]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
rectangular

0 12610.543630337896
5 12565.155364155204
7 12562.993289673057
18 12496.105202434834
36 12473.403697098916
171 12457.917408027963
max:  116 12782.740795328997
range:  324.823387301034


In [12]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
rectangular

0 14789.4765095688
2 14727.887681382774
29 14686.965786698609
40 14661.766532061534
204 14631.948573469024
max:  85 15144.512376700339
range:  512.5638032313145


In [13]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
rectangular

0 12608.004785461719
1 12464.713331198527
19 12446.671691899022
121 12441.05574558139
217 12401.2775066187
223 12393.217868670417
max:  160 12722.141903565638
range:  328.92403489522076


In [14]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
rectangular

0 13360.829181359934
2 13177.351186300937
4 13169.489021414205
5 13104.039519364684
21 13062.559532576797
67 13046.148271482034
117 13039.805080714985
189 13027.653892864077
221 12966.657510876159
max:  155 13664.056520633387
range:  697.3990097572278


In [15]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
rectangular

0 12651.992542904052
2 12509.251203238719
31 12483.80480487733
max:  22 12799.859415234643
range:  316.0546103573124


In [16]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
rectangular

0 14842.047278558242
1 14706.491968505572
20 14705.935741667161
39 14652.352200989266
82 14633.875745550838
240 14622.583041838152
max:  129 15006.071570855964
range:  383.48852901781174


In [17]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
rectangular

0 12551.999606176441
2 12543.744749562347
5 12477.05617432852
32 12455.505496940585
141 12453.53899461987
231 12446.266094003773
max:  51 12827.968735085746
range:  381.7026410819726


In [18]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
rectangular

0 13852.855198196014
1 13749.170720455857
2 13618.946734643307
3 13520.25745254452
12 13423.225831858153
56 13358.7208487047
max:  11 14050.497321335699
range:  691.7764726309997


In [19]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
rectangular

0 12617.537198538444
3 12587.289616437494
5 12495.920944850515
255 12481.48485752649
max:  174 12806.81220821146
range:  325.32735068497095


In [20]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
rectangular

0 14893.869933723006
1 14792.502194742618
2 14702.134992001786
73 14662.363965145976
96 14633.430597602543
max:  169 14981.084011836101
range:  347.6534142335586


In [21]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
rectangular

0 12588.300580862104
1 12574.265145121735
4 12554.863703018857
6 12549.841516959843
13 12483.36621570776
55 12427.28494870704
121 12423.156673357715
237 12422.511040814157
max:  119 12756.657768620062
range:  334.14672780590445


In [22]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
rectangular

0 13783.064744029385
1 13477.105912804132
26 13450.531584098737
33 13417.608674451563
38 13411.047601110142
80 13405.714059397054
108 13289.92586296366
max:  99 14014.585859277868
range:  724.6599963142071


In [23]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
hexagonal

0 12847.077367882825
2 12618.508018961476
13 12580.67941566078
195 12574.040687087418
max:  239 13021.357895112878
range:  447.3172080254608


In [24]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
hexagonal

0 14808.209650039496
9 14727.977744991276
102 14714.469614124617
max:  159 15158.591961807328
range:  444.1223476827108


In [25]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
hexagonal

0 12793.781481921438
2 12667.558081333475
4 12665.174503199198
13 12651.750778528078
16 12651.40400918912
27 12623.040564541316
76 12576.306826124173
91 12565.202819129752
195 12502.230011290643
max:  155 12911.291534157273
range:  409.0615228666302


In [26]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
hexagonal

0 13312.710052608136
2 13148.351216363375
7 13080.799714133704
52 13080.77861072666
62 13075.506133063836
74 13002.583659291977
100 12981.783542920222
214 12975.046199308103
max:  6 13952.112370589763
range:  977.06617128166


In [27]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
hexagonal

0 12713.690672010634
3 12681.580345746284
72 12637.14169238753
77 12629.278556166108
104 12610.79575739576
max:  149 12935.96180114036
range:  325.16604374460076


In [28]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
hexagonal

0 14820.800988298628
2 14764.986332405322
29 14764.399998836816
40 14719.582113787737
79 14698.096956699645
max:  85 15086.245935987929
range:  388.14897928828395


In [29]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
hexagonal

0 12706.349055786066
3 12705.848993474547
4 12682.919950499685
7 12601.46834144055
19 12565.717311011582
101 12515.30363457594
max:  230 12917.542544061705
range:  402.23890948576627


In [30]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
hexagonal

0 13333.947410851351
3 13300.985049701605
4 13137.796094379672
5 13117.215296978355
7 13064.204771450799
21 12997.995634792394
221 12975.578384299964
max:  168 13773.125939551195
range:  797.5475552512307


In [31]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
hexagonal

0 12730.087198170753
10 12660.646116721598
68 12648.503771905454
231 12606.626656944689
max:  189 12989.790190737838
range:  383.1635337931493


In [32]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
hexagonal

0 14894.53157898887
1 14806.983595262236
14 14806.541610303906
20 14787.660257823325
26 14778.502813380732
37 14764.815165808275
51 14722.486376652918
max:  190 15099.111643758642
range:  376.62526710572456


In [33]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
hexagonal

0 12788.880526620933
1 12772.502108864832
2 12638.205275672959
6 12610.04801385272
32 12600.366685124472
34 12588.894597586823
199 12565.679305816566
max:  52 12945.39292847063
range:  379.71362265406424


In [34]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
hexagonal

0 13824.31166865094
2 13735.073873060232
3 13536.12854029495
12 13459.146436925741
13 13435.929280979715
56 13362.365479791657
max:  179 14069.197147335562
range:  706.8316675439055


In [35]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
hexagonal

0 12781.759571770586
1 12670.484510467782
5 12642.662599574953
28 12622.268957475651
64 12599.817729196635
max:  92 12907.136294757513
range:  307.3185655608777


In [36]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
hexagonal

0 14823.717579289321
2 14800.450707386306
21 14773.616037126234
38 14727.460515550421
max:  32 15066.357043252563
range:  338.8965277021416


In [37]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
hexagonal

0 12704.298171909748
1 12683.698974864772
5 12642.473913958333
7 12633.265560772157
15 12620.93816107469
25 12589.030822396067
28 12572.353706157535
231 12492.766724369341
max:  8 12938.611055746023
range:  445.844331376682


In [38]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 4, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
hexagonal

0 13743.789285516385
1 13521.502511215573
13 13512.811954596807
26 13491.120949412178
33 13405.433611102882
108 13291.483157700255
max:  99 14137.646937528287
range:  846.1637798280317


In [39]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
rectangular

0 13517.702925589694
2 13405.459174286298
3 13314.159250643284
10 13286.157477494207
128 13283.955105741497
max:  7 13663.631634970818
range:  379.6765292293203


In [40]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
rectangular

0 14939.536903882446
1 14824.837623794367
14 14819.380730772084
33 14781.31403166079
127 14745.672979162151
max:  238 15212.236116838483
range:  466.5631376763322


In [41]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
rectangular

0 13496.952765025339
2 13416.098609108614
3 13412.127766756636
9 13400.855060889675
10 13311.105179291993
12 13302.302425475198
14 13247.835836958644
72 13238.789696016116
max:  239 13655.770772789798
range:  416.9810767736817


In [42]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
rectangular

0 13933.128963834892
1 13880.750946922442
5 13811.741706065768
7 13666.234600419704
15 13546.90053782095
229 13538.619831721056
max:  16 14249.128416534908
range:  710.5085848138515


In [43]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
rectangular

0 13428.72006846994
1 13377.54370922258
4 13358.191448267038
18 13354.579691962923
21 13353.294650765003
56 13350.928776229217
63 13339.09721520761
85 13334.124627052988
160 13300.32371718824
max:  81 13647.155360398565
range:  346.83164321032564


In [44]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
rectangular

0 14974.749571664745
1 14872.769169178762
11 14867.166335557333
13 14865.145968040684
14 14851.299348590472
16 14806.355014509507
33 14805.277863321351
71 14804.682562704586
77 14782.68529228453
max:  114 15169.332478630551
range:  386.64718634602104


In [45]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
rectangular

0 13367.807516440467
1 13356.323100374411
6 13353.823475185034
10 13352.855587587934
17 13335.111853624741
21 13281.305195031318
52 13241.349206888553
max:  157 13691.931698186247
range:  450.5824912976932


In [46]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
rectangular

0 14024.232201314842
1 13859.703301666044
2 13815.667206202741
3 13617.41994246455
9 13507.110942055773
max:  249 14421.066150378538
range:  913.9552083227645


In [47]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
rectangular

0 13356.089188792193
14 13341.543520090056
60 13294.574866245006
max:  184 13769.759968961489
range:  475.185102716483


In [48]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
rectangular

0 15101.918854566695
1 14972.757515454734
2 14895.547756924123
3 14804.34254064489
28 14789.427714873545
65 14764.227001277497
180 14718.213176424086
max:  12 15214.989773185836
range:  496.7765967617506


In [49]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
rectangular

0 13244.811525867446
max:  176 13671.65251200231
range:  426.8409861348646


In [50]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
rectangular

0 13789.483160724354
47 13745.381315833767
56 13639.941026231518
max:  236 14430.059209606536
range:  790.1181833750179


In [51]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
rectangular

0 13526.55257558994
1 13463.04061043302
4 13396.542564533189
10 13346.885659718126
14 13334.104262413237
19 13307.139449176637
119 13287.125624373777
159 13284.815252292135
177 13281.189912100846
max:  73 13820.31175504165
range:  539.1218429408036


In [52]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
rectangular

0 14860.130115611595
5 14841.604402890282
12 14817.44714144869
16 14763.2453011197
103 14744.083460686998
max:  83 15179.952425814668
range:  435.8689651276709


In [53]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
rectangular

0 13420.199166835526
8 13403.236854834671
11 13370.55952369743
16 13361.9092386029
19 13267.03824309695
188 13262.816709587614
max:  222 13686.297978632932
range:  423.4812690453182


In [54]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
rectangular

0 14024.04690642481
3 14023.601309841653
4 13895.872995165073
12 13890.136778368642
17 13879.744431008616
21 13865.308374945726
22 13853.120549810179
32 13792.478296865222
59 13780.870013582884
91 13712.621226179492
229 13596.234592858982
max:  242 14557.715171960795
range:  961.4805791018134


In [55]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
hexagonal

0 13536.370600835226
8 13516.717931653971
10 13509.664302222503
12 13442.887700178633
14 13320.112247042312
max:  239 13805.992398053355
range:  485.88015101104247


In [56]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
hexagonal

0 15071.5839369357
1 14811.632698310706
33 14775.439288417589
127 14752.17171110506
max:  238 15195.011750790516
range:  442.84003968545585


In [57]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
hexagonal

0 13495.303195007284
2 13449.915558993272
10 13336.418326007264
max:  239 13828.165773097815
range:  491.74744709055085


In [58]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
hexagonal

0 13933.130937455358
1 13908.886637771
4 13846.512453430534
5 13760.165840093034
7 13681.524012066879
15 13565.977733990552
71 13538.141269054122
max:  16 14224.078143235978
range:  685.9368741818562


In [59]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
hexagonal

0 13482.132461879604
1 13461.629172326146
4 13423.105605332343
56 13363.405980517728
max:  35 13762.298744750504
range:  398.89276423277624


In [60]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
hexagonal

0 14946.831617191872
1 14833.35525311441
14 14790.369146622128
19 14789.139185928405
max:  94 15205.863899131247
range:  416.7247132028424


In [61]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
hexagonal

0 13511.17477651932
2 13494.252910777082
4 13400.375060029282
17 13377.794055306142
199 13324.371189972333
max:  81 13814.322673418115
range:  489.9514834457823


In [62]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
hexagonal

0 14002.902589535439
1 13837.435249458707
3 13644.249139849042
9 13557.094848088149
242 13530.578603257662
max:  249 14297.463120253109
range:  766.8845169954475


In [63]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
hexagonal

0 13488.909052876788
6 13486.97075564617
14 13405.310181290268
44 13379.709545718511
max:  184 13805.44955376345
range:  425.7400080449388


In [64]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
hexagonal

0 14835.167921267093
3 14822.638778245428
28 14793.68057892836
53 14790.813969980616
61 14781.84459929924
65 14761.372199325686
180 14739.535492456473
max:  33 15263.354316269377
range:  523.8188238129042


In [65]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
hexagonal

0 13407.286069613676
54 13365.430397618986
max:  184 13776.316915967902
range:  410.88651834891607


In [66]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
hexagonal

0 13881.579364879704
13 13878.915305146815
26 13832.989852762059
56 13670.596904237962
max:  236 14679.754641230984
range:  1009.1577369930219


In [67]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
hexagonal

0 13574.258127863372
1 13517.30919051427
3 13476.37897056874
14 13422.309628604347
100 13404.36667319993
188 13389.041546572938
max:  222 13798.414853638516
range:  409.37330706557805


In [68]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
hexagonal

0 14866.2435427627
5 14838.64996239808
16 14764.879034558453
103 14756.130220798988
max:  152 15108.283163265423
range:  352.152942466435


In [69]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
hexagonal

0 13592.391904856955
1 13543.726022503188
2 13496.163315200249
5 13469.083964521813
9 13458.550550019465
15 13446.831747956925
25 13351.371617813369
max:  73 13830.06865352647
range:  478.6970357131013


In [70]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 3, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
hexagonal

0 14016.804273093434
4 13969.215589463596
11 13953.876954621408
15 13920.929967152186
17 13894.653911556665
21 13887.946921671013
32 13815.366752626654
91 13692.235567751666
max:  151 14537.677156303653
range:  845.4415885519866


In [71]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
rectangular

0 12107.055436445251
2 11978.519231200667
42 11975.994770740394
72 11971.309619438973
160 11936.335164148462
max:  169 12275.734023347062
range:  339.3988591985999


In [72]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
rectangular

0 14830.840795297956
1 14703.768287545772
9 14702.316137721631
22 14697.610005938506
25 14605.492691347357
37 14586.188169835808
48 14581.280113813302
250 14505.49988349986
max:  172 14998.171673719227
range:  492.6717902193668


In [73]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
rectangular

0 12003.778690010404
1 11955.913389263173
2 11799.086284501074
max:  169 12098.027037235044
range:  298.94075273396993


In [74]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
rectangular

0 12890.257540675308
2 12569.974897157792
45 12532.378394121246
max:  101 13200.699391649414
range:  668.3209975281679


In [75]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
rectangular

0 12021.635626900284
8 12004.188663890474
15 11967.796364506194
73 11937.2184893207
max:  165 12185.768323558525
range:  248.54983423782505


In [76]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
rectangular

0 14886.110833321101
1 14739.521989489695
2 14685.030762540773
23 14635.468177386192
29 14607.34151981415
42 14588.581174578114
240 14569.66176899451
max:  39 15039.372468003445
range:  469.71069900893417


In [77]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
rectangular

0 11965.956418587597
1 11934.08741042031
2 11911.387405726478
4 11889.907926468793
29 11875.88850114214
46 11875.697996136903
67 11845.850625217103
101 11838.389191431566
166 11807.974126542807
max:  135 12082.288278190803
range:  274.31415164799546


In [78]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
rectangular

0 13022.104329161835
1 12905.780603672494
2 12758.447927357116
4 12727.248473433287
6 12686.626418452537
13 12650.008464791614
47 12605.69728443785
151 12562.842181148357
max:  101 13276.147526678616
range:  713.3053455302597


In [79]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
rectangular

0 12097.402628748257
1 12032.624907108855
2 12011.137240186834
7 11998.069376729023
12 11993.70975616168
14 11969.109825553409
190 11933.267931369046
max:  177 12266.726487226491
range:  333.4585558574454


In [80]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
rectangular

0 14826.76745691405
1 14685.794737327644
2 14672.461909807082
7 14652.139385244454
17 14642.080228339135
27 14619.791829421118
109 14562.32600734367
153 14547.67278198804
max:  161 15035.00563213028
range:  487.3328501422402


In [81]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
rectangular

0 11945.543834982516
2 11925.47962904672
7 11916.4922217363
12 11851.732995284498
151 11832.595233009843
max:  111 12146.914198500097
range:  314.3189654902544


In [82]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
rectangular

0 13452.951587390578
1 13402.906619833964
2 13378.64815070108
3 13256.962167858725
6 13220.412951083374
12 13213.284143930747
30 13180.815349714287
41 13180.74456027327
55 13071.052787825096
max:  77 13751.21286653236
range:  680.1600787072639


In [83]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
rectangular

0 12087.767898301865
1 12012.208568322485
2 11983.744172319526
13 11974.746110030106
98 11927.15150345913
max:  160 12230.858528199715
range:  303.70702474058453


In [84]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
rectangular

0 14702.878202096677
4 14670.885460801714
9 14654.081441816492
20 14631.349881086857
25 14618.799052064653
62 14609.955517990953
160 14598.187337865684
max:  238 14982.768149314235
range:  384.5808114485517


In [85]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
rectangular

0 12016.495133949837
1 11875.209503788159
3 11843.770570814242
12 11810.17647788879
max:  221 12055.165827668932
range:  244.98934978014222


In [86]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "rectangular\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "rectangular", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
rectangular

0 13473.018199608268
2 13398.191340675616
3 13215.406727299656
42 13099.9780188374
max:  114 13684.812089870818
range:  584.8340710334178


In [87]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
stochastic
hexagonal

0 12251.92880403605
1 12243.404209527154
2 12124.456048395648
14 12091.900920192158
129 12087.66585201844
242 12049.988737415893
max:  212 12395.890095993014
range:  345.90135857712085


In [88]:
print("prop radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
stochastic
hexagonal

0 14847.155803593712
1 14732.291300232106
5 14715.914597626377
9 14703.13233235824
23 14653.075374199985
37 14652.421662558041
48 14605.023819441945
52 14592.587921393653
250 14482.611191165715
max:  100 15025.222569969528
range:  542.6113788038128


In [89]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
stochastic
hexagonal

0 12205.104999408435
1 12164.589058839825
2 12044.073655093456
5 12031.050574571644
10 11994.84880897234
48 11988.652618403084
91 11975.305984428458
max:  212 12291.77117493584
range:  316.46519050738243


In [90]:
print("exp radius\n", "prop learning rate\n", "sample\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
stochastic
hexagonal

0 12859.413869617169
2 12586.366950640298
45 12534.046342537675
max:  101 13146.892679228577
range:  612.8463366909018


In [91]:
print("prop radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
sample
random shuffle
hexagonal

0 12199.464361323002
2 12175.466775796565
4 12174.346814545372
8 12149.288511097324
12 12147.02672373289
14 12119.13881824522
20 12097.769638373908
73 12074.141877646669
142 12060.736674356316
max:  10 12341.715712095684
range:  280.9790377393674


In [92]:
print("prop radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
sample
random shuffle
hexagonal

0 14924.579753570688
1 14773.49414689709
2 14681.53373796943
23 14673.126446895934
29 14590.148320424756
240 14585.558030354588
max:  193 15095.073426279361
range:  509.51539592477275


In [93]:
print("exp radius\n", "fixed learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
sample
random shuffle
hexagonal

0 12085.807248543879
6 12081.51221852753
7 12044.968659244258
8 12009.394025485311
22 11988.698241316903
29 11965.548308806538
112 11926.09377859212
max:  141 12228.190898382472
range:  302.09711979035274


In [94]:
print("exp radius\n", "prop learning rate\n", "sample\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "sample no repos", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
sample
random shuffle
hexagonal

0 12965.444697038507
1 12904.275637663837
2 12720.046941123437
4 12718.79957257654
6 12637.115585688345
13 12603.887356287782
119 12538.082892766917
151 12523.594832230676
max:  101 13281.824749245166
range:  758.2299170144906


In [95]:
print("prop radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
stochastic
hexagonal

0 12203.408608629088
1 12177.778000773045
2 12166.112691059596
3 12139.687354746184
19 12132.149692987394
21 12114.732041109568
42 12107.401751945545
51 12106.036910614654
99 12079.35482881935
max:  72 12362.280478306799
range:  282.925649487448


In [96]:
print("prop radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
stochastic
hexagonal

0 14861.996189801359
1 14721.641190083756
2 14704.408481257587
7 14642.686775594415
27 14616.915250406906
109 14608.701723575072
153 14549.665801276393
max:  166 15008.705061092427
range:  459.03925981603425


In [97]:
print("exp radius\n", "fixed learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
stochastic
hexagonal

0 12147.981617632231
1 12072.416831817261
2 12062.6427542474
7 12036.721985586984
37 12008.974333228036
86 11959.208479975307
max:  131 12275.111334134825
range:  315.90285415951803


In [98]:
print("exp radius\n", "prop learning rate\n", "random\n", "stochastic\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "stochastic", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
stochastic
hexagonal

0 13451.282588287757
1 13394.060255729572
3 13323.07542558255
6 13236.197568884223
12 13198.5885878247
30 13183.702072179476
55 13022.605112147168
max:  77 13715.307821073035
range:  692.7027089258663


In [99]:
print("prop radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
fixed learning rate
random
random shuffle
hexagonal

0 12210.025387105088
1 12105.539438756547
56 12097.185994103573
69 12095.224691891057
98 12084.969648679398
188 12076.90523681758
197 12060.256409256197
max:  140 12372.583630411462
range:  312.32722115526485


In [100]:
print("prop radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    prop2iters_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

prop radius
prop learning rate
random
random shuffle
hexagonal

0 14704.50386731386
1 14679.769770423125
4 14675.564299174413
13 14670.475267543172
16 14643.235178912239
20 14641.063663823483
25 14591.989637544206
max:  221 15030.813901982045
range:  438.8242644378388


In [101]:
print("exp radius\n", "fixed learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, fixed_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
fixed learning rate
random
random shuffle
hexagonal

0 12223.62691804466
1 12014.71083182996
3 12007.348629991498
18 11992.364589548646
112 11985.638665124696
208 11984.240193289977
236 11978.831827351585
max:  221 12230.497977539759
range:  251.66615018817356


In [102]:
print("exp radius\n", "prop learning rate\n", "random\n", "random shuffle\n", "hexagonal\n", sep="")
min_metric = None
max_metric = None
min_kohonen_seed = None
max_kohonen_seed = None
for seed in range(seed_range):
    random_state = np.random.default_rng(seed)
    kohonen_net = build_kohonen_net(df, cols, 5, len(cols)*500, 
                                    "random", "random shuffle", 
                                    exp_neighbour_radius, inv2iter_learning_rate, 
                                    "hexagonal", random_state, True, len(cols)*500)

    if min_metric is None or min_metric > kohonen_net.sum_dists[-1][1]:
        min_metric = kohonen_net.sum_dists[-1][1]
        min_kohonen_seed = seed
        print(min_kohonen_seed, min_metric)

    if max_metric is None or max_metric < kohonen_net.sum_dists[-1][1]:
        max_metric = kohonen_net.sum_dists[-1][1]
        max_kohonen_seed = seed

print("max: ", max_kohonen_seed, max_metric)
print("range: ", max_metric - min_metric)

exp radius
prop learning rate
random
random shuffle
hexagonal

0 13405.349476437592
2 13383.884276504998
3 13207.894851857895
42 13082.480248666372
max:  106 13704.36576573448
range:  621.8855170681072
